# Look at raw data

In [26]:
from datasets import load_dataset, get_dataset_config_names

def load_hf_dataset(repo_id):
  configs = []
  try:
    configs = get_dataset_config_names(repo_id)
  except Exception:
    pass

  if configs:
    print(f'{repo_id} configs: {configs}')
    ds = load_dataset(repo_id, configs[0])
  else:
    ds = load_dataset(repo_id)

  return ds

def print_rows(ds, n=5):
  if hasattr(ds, 'keys'):
    splits = list(ds.keys())
    print(f'splits: {splits}')
    for split in splits:
      d = ds[split]
      print(f'\n--- split: {split} | rows: {len(d)} | columns: {d.column_names} ---')
      for i in range(min(n, len(d))):
        print(d[i])
  else:
    print(f'rows: {len(ds)} | columns: {ds.column_names}')
    for i in range(min(n, len(ds))):
      print(ds[i])

In [27]:
romb = load_hf_dataset('d0rj/ROMB-1.0')
print_rows(romb, n=5)

d0rj/ROMB-1.0 configs: ['default']
splits: ['test']

--- split: test | rows: 2552 | columns: ['id', 'task_text', 'answer_text', 'correct_answer', 'date', 'olymp_name', 'grade', 'description', 'source', 'answer_type', 'check_type', 'check_function', 'task_type', 'task_note'] ---
{'id': 0, 'task_text': 'Если a и b — натуральные числа, ни одно из которых не делится на 10, и ab = 10000, то a + b равно:\n(А) 1024\n(Б) 641\n(В) 74\n(Г) 34\n(Д) 1000', 'answer_text': 'Ответ: Б', 'correct_answer': '"Б"', 'date': '18 марта 2004', 'olymp_name': 'Международный конкурс по математике Кенгуру', 'grade': '7-8', 'description': 'Международный конкурс по математике Кенгуру, 7-8 класс, 2004 год', 'source': 'https://ipokengu.ru/konkurs-kenguru/zadachi.html', 'answer_type': "Literal['А', 'Б', 'В', 'Г', 'Д']", 'check_type': 'em', 'check_function': None, 'task_type': 'arith', 'task_note': 'Ответ должен быть буквой правильного варианта ответа.'}
{'id': 1, 'task_text': 'В очереди за мороженым Катя стоит третья,

In [28]:
gsm8k_ru = load_hf_dataset('d0rj/gsm8k-ru')
print_rows(gsm8k_ru, n=5)

d0rj/gsm8k-ru configs: ['default']
splits: ['train', 'test']

--- split: train | rows: 7473 | columns: ['question', 'answer'] ---
{'question': 'В апреле Наталья продала клипы 48 своим друзьям, а в мае продала вдвое меньше клипов. Сколько всего клипов продала Наталья в апреле и мае?', 'answer': 'Наталья продала 48/2 = <<48/2=24>>24 ролика в мае.\nВсего за апрель и май Наталья продала 48+24 = <<48+24=72>>72 клипа.\n#### 72'}
{'question': 'Венг зарабатывает 12 долларов в час за присмотр за детьми. Вчера она всего 50 минут сидела с детьми. Сколько она заработала?', 'answer': 'Венг зарабатывает 12/60 = $<<12/60=0,2>>0,2 в минуту.\nРаботая 50 минут, она заработала 0,2 x 50 = $<<0,2*50=10>>10.\n#### 10'}
{'question': 'Бетти копит деньги на новый бумажник, который стоит 100 долларов. У Бетти есть только половина денег, которые ей нужны. Ее родители решили дать ей на эти цели 15 долларов, а бабушка и дедушка — в два раза больше, чем ее родители. Сколько еще денег нужно Бетти, чтобы купить бумаж

In [29]:
t_math = load_hf_dataset('t-tech/T-math')
print_rows(t_math, n=5)

t-tech/T-math configs: ['default']
splits: ['train']

--- split: train | rows: 331 | columns: ['question', 'verifiable_answer', 'year', 'grade', 'full_answer', 'solutions', 'task_complexity', 'olympiad'] ---
{'question': 'На острове Невезения с населением 96 человек провели пять реформ. Каждой реформой недовольна ровно половина граждан. Гражданин выходит на митинг, если недоволен более чем половиной реформ. Какое максимальное число людей может быть на митинге?', 'verifiable_answer': '80', 'year': '2005', 'grade': 'all', 'full_answer': '80', 'solutions': ['Общее число недовольств: $ 48 \\cdot 5 = 240 $. Каждый на митинге недоволен хотя бы тремя реформами, поэтому $ 3x \\leq 240 $, $ x \\leq 80 $. Пример: 80 человек, разбитых на 5 групп по 16, недовольных разными реформами.'], 'task_complexity': 'MEDIUM', 'olympiad': 'mos'}
{'question': 'Клетчатый квадрат $8 \\times 8$ согнули по линиям клеток в квадрат $1 \\times 1$. Его разрезали по отрезку, соединяющему середины двух противоположных с

# Unite and prepare data to single HF dataset

In [30]:
from datasets import concatenate_datasets, Dataset, DatasetDict

romb = load_dataset('d0rj/ROMB-1.0', split='test')
gsm_train = load_dataset('d0rj/gsm8k-ru', split='train')
gsm_test = load_dataset('d0rj/gsm8k-ru', split='test')
tmath = load_dataset('t-tech/T-math', split='train')

print(len(romb), len(gsm_train), len(gsm_test), len(tmath))

2552 7473 1319 331


In [31]:
import re

def _strip_wrapping_quotes(s):
  if s is None:
    return ''
  s = str(s).strip()
  if len(s) >= 2 and ((s[0] == '"' and s[-1] == '"') or (s[0] == "'" and s[-1] == "'")):
    s = s[1:-1].strip()
  return s

def _make_text(prompt, response):
  prompt = (prompt or '').strip()
  response = (response or '').strip()
  return (
    '### Задание:\nРеши задачу по математике.\n\n'
    f'### Условие:\n{prompt}\n\n'
    f'### Ответ:\n{response}'
  )

def map_romb(x):
  q = x.get('task_text', '')
  a = x.get('answer_text') or x.get('correct_answer') or ''
  a = _strip_wrapping_quotes(a)
  prompt = q
  response = a
  return {
    'source': 'd0rj/ROMB-1.0',
    'prompt': prompt,
    'response': response,
    'text': _make_text(prompt, response),
  }

def map_gsm(x):
  q = x.get('question', '')
  a = x.get('answer', '')
  prompt = q
  response = a
  return {
    'source': 'd0rj/gsm8k-ru',
    'prompt': prompt,
    'response': response,
    'text': _make_text(prompt, response),
  }

def map_tmath(x):
  q = x.get('question', '')
  sols = x.get('solutions') or []
  if isinstance(sols, list) and len(sols) > 0:
    a = '\n\n'.join([str(s).strip() for s in sols if str(s).strip()])
  else:
    a = x.get('full_answer') or x.get('verifiable_answer') or ''
  prompt = q
  response = str(a).strip()
  return {
    'source': 't-tech/T-math',
    'prompt': prompt,
    'response': response,
    'text': _make_text(prompt, response),
  }

romb_sft = romb.map(map_romb, remove_columns=romb.column_names)
gsm_sft = concatenate_datasets([
  gsm_train.map(map_gsm, remove_columns=gsm_train.column_names),
  gsm_test.map(map_gsm, remove_columns=gsm_test.column_names),
])
tmath_sft = tmath.map(map_tmath, remove_columns=tmath.column_names)

print(len(romb_sft), len(gsm_sft), len(tmath_sft))
print(romb_sft[0])

2552 8792 331
{'source': 'd0rj/ROMB-1.0', 'prompt': 'Если a и b — натуральные числа, ни одно из которых не делится на 10, и ab = 10000, то a + b равно:\n(А) 1024\n(Б) 641\n(В) 74\n(Г) 34\n(Д) 1000', 'response': 'Ответ: Б', 'text': '### Задание:\nРеши задачу по математике.\n\n### Условие:\nЕсли a и b — натуральные числа, ни одно из которых не делится на 10, и ab = 10000, то a + b равно:\n(А) 1024\n(Б) 641\n(В) 74\n(Г) 34\n(Д) 1000\n\n### Ответ:\nОтвет: Б'}


In [32]:
full = concatenate_datasets([romb_sft, gsm_sft, tmath_sft]).shuffle(seed=42)

splits = full.train_test_split(test_size=0.1, seed=42)
ds = DatasetDict({
  'train': splits['train'],
  'test': splits['test'],
})

print(ds)
print('train:', len(ds['train']), 'test:', len(ds['test']))

DatasetDict({
    train: Dataset({
        features: ['source', 'prompt', 'response', 'text'],
        num_rows: 10507
    })
    test: Dataset({
        features: ['source', 'prompt', 'response', 'text'],
        num_rows: 1168
    })
})
train: 10507 test: 1168


In [33]:
ds['test'][3]

{'source': 'd0rj/ROMB-1.0',
 'prompt': 'Каждый пират либо всегда лжёт, либо всегда говорит правду. Купец же может сказать что угодно когда угодно. Как-то раз Хью зашёл в таверну и услышал, как три человека, кричали друг другу следующее.\n- Первый второму: «Ты только что солгал!»\n- Второй третьему: «Ты только что солгал!»\n- Третий первому: «Ты только что солгал!»\n- Первый второму: «Ты только что солгал!»\n- Второй третьему: «Ты только что солгал!»\n\nКакое наименьшее количество купцов могло быть среди этих троих?',
 'response': 'Ответ: 1',
 'text': '### Задание:\nРеши задачу по математике.\n\n### Условие:\nКаждый пират либо всегда лжёт, либо всегда говорит правду. Купец же может сказать что угодно когда угодно. Как-то раз Хью зашёл в таверну и услышал, как три человека, кричали друг другу следующее.\n- Первый второму: «Ты только что солгал!»\n- Второй третьему: «Ты только что солгал!»\n- Третий первому: «Ты только что солгал!»\n- Первый второму: «Ты только что солгал!»\n- Второй трет

In [34]:
ds['test'][10]

{'source': 'd0rj/gsm8k-ru',
 'prompt': 'У Майкла 2 кошки и 3 собаки. Ему нужно заплатить другу, чтобы тот посмотрел на них, который берет 13 долларов за ночь за животное. Сколько должен заплатить Михаил?',
 'response': 'У Майкла 5 животных, потому что 2 + 3 = <<2+3=5>>5\nМайкл должен заплатить 65 долларов, потому что 5 x 13 долларов = $<<5*13=65>>65.\n#### 65',
 'text': '### Задание:\nРеши задачу по математике.\n\n### Условие:\nУ Майкла 2 кошки и 3 собаки. Ему нужно заплатить другу, чтобы тот посмотрел на них, который берет 13 долларов за ночь за животное. Сколько должен заплатить Михаил?\n\n### Ответ:\nУ Майкла 5 животных, потому что 2 + 3 = <<2+3=5>>5\nМайкл должен заплатить 65 долларов, потому что 5 x 13 долларов = $<<5*13=65>>65.\n#### 65'}

In [35]:
ds['train'][100]

{'source': 'd0rj/gsm8k-ru',
 'prompt': 'Рональд бросает кубик и не останавливается, пока среднее значение всех его бросков не станет 3. Он кидает 1, 3, 2, 4, 3, 5, 3, 4, 4. и 2. Что ему нужно сделать при следующем броске, чтобы он смог остановиться?',
 'response': 'Следующий бросок будет его 11-м броском.\nОбщее количество на этом десятом броске, чтобы получить в среднем 3, равно 33, потому что 3 * 11 = <<33=33>>33.\nЕго текущая сумма равна 31, потому что 1 + 3 + 2 + 4 + 3 + 5 + 6 + 1 + 4 + 2= <<1+3+2+4+3+5+6+1+4+2=31> >31\nЕму нужно выбросить 2, потому что 33 - 31 = <<33-31=2>>2\n#### 2',
 'text': '### Задание:\nРеши задачу по математике.\n\n### Условие:\nРональд бросает кубик и не останавливается, пока среднее значение всех его бросков не станет 3. Он кидает 1, 3, 2, 4, 3, 5, 3, 4, 4. и 2. Что ему нужно сделать при следующем броске, чтобы он смог остановиться?\n\n### Ответ:\nСледующий бросок будет его 11-м броском.\nОбщее количество на этом десятом броске, чтобы получить в среднем 3,

In [36]:
def add_messages(row):
  return {
    'messages': [
      {'role': 'user', 'content': row['prompt']},
      {'role': 'assistant', 'content': row['response']}
    ]
  }

ds = ds.map(add_messages)

In [38]:
ds['train'][100]

{'source': 'd0rj/gsm8k-ru',
 'prompt': 'Рональд бросает кубик и не останавливается, пока среднее значение всех его бросков не станет 3. Он кидает 1, 3, 2, 4, 3, 5, 3, 4, 4. и 2. Что ему нужно сделать при следующем броске, чтобы он смог остановиться?',
 'response': 'Следующий бросок будет его 11-м броском.\nОбщее количество на этом десятом броске, чтобы получить в среднем 3, равно 33, потому что 3 * 11 = <<33=33>>33.\nЕго текущая сумма равна 31, потому что 1 + 3 + 2 + 4 + 3 + 5 + 6 + 1 + 4 + 2= <<1+3+2+4+3+5+6+1+4+2=31> >31\nЕму нужно выбросить 2, потому что 33 - 31 = <<33-31=2>>2\n#### 2',
 'text': '### Задание:\nРеши задачу по математике.\n\n### Условие:\nРональд бросает кубик и не останавливается, пока среднее значение всех его бросков не станет 3. Он кидает 1, 3, 2, 4, 3, 5, 3, 4, 4. и 2. Что ему нужно сделать при следующем броске, чтобы он смог остановиться?\n\n### Ответ:\nСледующий бросок будет его 11-м броском.\nОбщее количество на этом десятом броске, чтобы получить в среднем 3,

In [ ]:
ds.save_to_disk('dataset_math_raw')

Saving the dataset (0/1 shards):   0%|          | 0/10507 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1168 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_from_disk

ch = load_from_disk('dataset_math_raw')
ch['train'][100]

{'source': 'd0rj/gsm8k-ru',
 'prompt': 'Рональд бросает кубик и не останавливается, пока среднее значение всех его бросков не станет 3. Он кидает 1, 3, 2, 4, 3, 5, 3, 4, 4. и 2. Что ему нужно сделать при следующем броске, чтобы он смог остановиться?',
 'response': 'Следующий бросок будет его 11-м броском.\nОбщее количество на этом десятом броске, чтобы получить в среднем 3, равно 33, потому что 3 * 11 = <<33=33>>33.\nЕго текущая сумма равна 31, потому что 1 + 3 + 2 + 4 + 3 + 5 + 6 + 1 + 4 + 2= <<1+3+2+4+3+5+6+1+4+2=31> >31\nЕму нужно выбросить 2, потому что 33 - 31 = <<33-31=2>>2\n#### 2',
 'text': '### Задание:\nРеши задачу по математике.\n\n### Условие:\nРональд бросает кубик и не останавливается, пока среднее значение всех его бросков не станет 3. Он кидает 1, 3, 2, 4, 3, 5, 3, 4, 4. и 2. Что ему нужно сделать при следующем броске, чтобы он смог остановиться?\n\n### Ответ:\nСледующий бросок будет его 11-м броском.\nОбщее количество на этом десятом броске, чтобы получить в среднем 3,

In [44]:
from datasets import load_from_disk
import os

src = 'dataset_math_raw'
out = 'dataset_math_messages_jsonl'
os.makedirs(out, exist_ok=True)

ds = load_from_disk(src)

ds = ds.remove_columns([c for c in ds['train'].column_names if c != 'messages'])

ds['train'].to_json(f'{out}/train.jsonl', orient='records', lines=True, force_ascii=False)
ds['test'].to_json(f'{out}/test.jsonl', orient='records', lines=True, force_ascii=False)
print('wrote', out)

Creating json from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

wrote /app/data/TEMP/genai/dataset_math_messages_jsonl
